In [1]:
# install
!pip install --upgrade openai langchain-openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0
  Attempting uninstall: langchain-core
    Found e

In [5]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = ""

llm_base = ChatOpenAI(model="gpt-4", temperature=0.7)

# Structured Output with Pydantic

In [ ]:
from pydantic import BaseModel, Field

class SearchTask(BaseModel):
    reformulated: str = Field(..., description="Optimized query for web search")
    rationale: str = Field(..., description="Why this query is effective")

structured_llm = llm_base.with_structured_output(SearchTask)
structured_out = structured_llm.invoke("What's the connection between gut health and mental health?")
print("Structured Output:\n", structured_out)

# Tool Binding

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(x: int, y: int) -> int:
    return x * y

@tool
def add(x: int, y: int) -> int:
    return x + y

@tool
def divide(x: int, y: int) -> float:
    return x / y

math_tools = [multiply, add, divide]
llm_with_math = llm_base.bind_tools(math_tools)

response = llm_with_math.invoke("What's 15 divided by 3?")
print("Tool Call Output:\n", response.tool_calls)

# Prompt Chaining

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class JokeState(TypedDict):
    topic: str
    raw_joke: str
    upgraded_joke: str
    final_joke: str

def make_joke(state: JokeState):
    return {"raw_joke": llm_base.invoke(f"Tell a joke about {state['topic']}").content}

def has_punchline(state: JokeState):
    return "good" if "!" in state["raw_joke"] or "?" in state["raw_joke"] else "edit"

def improve_joke(state: JokeState):
    return {"upgraded_joke": llm_base.invoke(f"Improve this joke: {state['raw_joke']}").content}

def polish_joke(state: JokeState):
    return {"final_joke": llm_base.invoke(f"Add a twist: {state['upgraded_joke']}").content}

In [ ]:
joke_graph = StateGraph(JokeState)
joke_graph.add_node("make_joke", make_joke)
joke_graph.add_node("improve_joke", improve_joke)
joke_graph.add_node("polish_joke", polish_joke)
joke_graph.add_edge(START, "make_joke")
joke_graph.add_conditional_edges("make_joke", has_punchline, {"good": END, "edit": "improve_joke"})
joke_graph.add_edge("improve_joke", "polish_joke")
joke_graph.add_edge("polish_joke", END)
joke_chain = joke_graph.compile()

result = joke_chain.invoke({"topic": "aliens"})
print("Final Joke Flow Result:\n", result)

# Parallel Graph

In [ ]:
class CreativeState(TypedDict):
    theme: str
    story: str
    haiku: str
    tweet: str
    combined: str

def gen_story(s): return {"story": llm_base.invoke(f"Write a short story about {s['theme']}").content}
def gen_haiku(s): return {"haiku": llm_base.invoke(f"Haiku on {s['theme']}").content}
def gen_tweet(s): return {"tweet": llm_base.invoke(f"Tweet about {s['theme']}").content}

def merge_outputs(s):
    return {
        "combined": f"📝 Story: {s['story'][:80]}...\n🌸 Haiku: {s['haiku']}\n🐦 Tweet: {s['tweet']}"
    }

In [ ]:
parallel = StateGraph(CreativeState)
parallel.add_node("gen_story", gen_story)
parallel.add_node("gen_haiku", gen_haiku)
parallel.add_node("gen_tweet", gen_tweet)
parallel.add_node("merge", merge_outputs)

parallel.add_edge(START, "gen_story")
parallel.add_edge(START, "gen_haiku")
parallel.add_edge(START, "gen_tweet")
parallel.add_edge("gen_story", "merge")
parallel.add_edge("gen_haiku", "merge")
parallel.add_edge("gen_tweet", "merge")
parallel.add_edge("merge", END)

creative_chain = parallel.compile()
out = creative_chain.invoke({"theme": "ocean"})
print("Creative Fusion Output:\n", out["combined"])

# Dynamic Router

In [ ]:
from typing import Literal
class Selector(BaseModel):
    route: Literal["story", "poem", "joke"] = Field(...)

router_llm = llm_base.with_structured_output(Selector)

class RouteState(TypedDict):
    input: str
    route: str
    result: str

def choose_route(s: RouteState):
    d = router_llm.invoke(f"Classify this: {s['input']}")
    return {"route": d.route}

def do_story(s): return {"result": llm_base.invoke(f"Story: {s['input']}").content}
def do_poem(s): return {"result": llm_base.invoke(f"Poem: {s['input']}").content}
def do_joke(s): return {"result": llm_base.invoke(f"Joke: {s['input']}").content}

def routing_logic(s: RouteState):
    return {"story": "do_story", "poem": "do_poem", "joke": "do_joke"}[s["route"]]

In [ ]:
route_graph = StateGraph(RouteState)
route_graph.add_node("choose_route", choose_route)
route_graph.add_node("do_story", do_story)
route_graph.add_node("do_poem", do_poem)
route_graph.add_node("do_joke", do_joke)

route_graph.add_edge(START, "choose_route")
route_graph.add_conditional_edges("choose_route", routing_logic,
    {"do_story": "do_story", "do_poem": "do_poem", "do_joke": "do_joke"})
route_graph.add_edge("do_story", END)
route_graph.add_edge("do_poem", END)
route_graph.add_edge("do_joke", END)

compiled_route = route_graph.compile()
out = compiled_route.invoke({"input": "Write me a short poem about dawn"})
print("Routing Output:\n", out["result"])

# Evaluator-Optimizer Loop

In [ ]:
class Review(BaseModel):
    verdict: Literal["funny", "not funny"]
    comments: str

from typing_extensions import Annotated
import operator

class FunnyState(TypedDict):
    topic: str
    joke: str
    verdict: str
    comments: str

eval_llm = llm_base.with_structured_output(Review)

In [ ]:
def generate_funny(s: FunnyState):
    if s.get("comments"):
        return {"joke": llm_base.invoke(f"Joke about {s['topic']} with feedback: {s['comments']}").content}
    return {"joke": llm_base.invoke(f"Joke about {s['topic']}").content}

def evaluate_funny(s: FunnyState):
    result = eval_llm.invoke(s["joke"])
    return {"verdict": result.verdict, "comments": result.comments}

def route_funny(s: FunnyState):
    return "pass" if s["verdict"] == "funny" else "retry"

In [ ]:
loop = StateGraph(FunnyState)
loop.add_node("generate", generate_funny)
loop.add_node("evaluate", evaluate_funny)
loop.add_edge(START, "generate")
loop.add_edge("generate", "evaluate")
loop.add_conditional_edges("evaluate", route_funny, {"pass": END, "retry": "generate"})
funny_chain = loop.compile()

out = funny_chain.invoke({"topic": "coffee"})
print("Final Joke:\n", out["joke"])

# Agent with Tool

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

tools_map = {t.name: t for t in math_tools}
llm_agent = llm_base.bind_tools(math_tools)

def agent_call(state: MessagesState):
    msg = llm_agent.invoke([SystemMessage(content="You are a math helper.")] + state["messages"])
    return {"messages": [msg]}

def run_tool(state: MessagesState):
    outputs = []
    for call in state["messages"][-1].tool_calls:
        fn = tools_map[call["name"]]
        result = fn.invoke(call["args"])
        outputs.append(ToolMessage(content=result, tool_call_id=call["id"]))
    return {"messages": outputs}

def should_continue(state: MessagesState):
    return "tool" if state["messages"][-1].tool_calls else END

In [ ]:
agent_graph = StateGraph(MessagesState)
agent_graph.add_node("agent_call", agent_call)
agent_graph.add_node("tool", run_tool)
agent_graph.add_edge(START, "agent_call")
agent_graph.add_conditional_edges("agent_call", should_continue, {"tool": "tool", END: END})
agent_graph.add_edge("tool", "agent_call")
agent_final = agent_graph.compile()

messages = agent_final.invoke({"messages": [HumanMessage(content="What is 8 * 7?")]})
for msg in messages["messages"]:
    msg.pretty_print()

# LangSmith Trace Example

In [ ]:
from langsmith import traceable

@traceable(run_type="llm")
def explain_topic(topic: str):
    return llm_base.invoke(f"Explain {topic} in simple terms.").content

print("LangSmith Trace:\n", explain_topic("blockchain"))